In [168]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.model_selection import RepeatedKFold
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from mlxtend.regressor import StackingCVRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
import optuna
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import Lasso, Ridge, ElasticNet, RANSACRegressor
from sklearn.base import clone
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

import pickle

In [169]:
df = pd.read_excel('../Data/Dataset.xlsx')
id = df[['id']]
df = df.drop('id', axis=1)
df

,POV,FOOD,ELEC,WATER,LIFE,HEALTH,SCHOOL,STUNTING,IKP
0,18.37,40.54,0.99,15.13,65.48,1.67,9.74,37.2,72.51
1,19.18,35.00,0.00,41.74,67.65,1.77,8.98,34.0,47.06
2,12.43,47.68,0.21,41.78,64.64,2.37,8.09,34.8,71.11
3,12.83,25.28,0.07,47.30,68.48,2.33,9.94,36.7,78.47
4,13.91,37.31,0.07,45.47,68.94,1.69,8.83,33.6,75.78
...,...,...,...,...,...,...,...,...,...
509,20.56,6.80,1.70,17.90,66.16,0.49,10.60,24.9,86.06
510,3.11,5.23,0.00,5.95,71.38,0.09,11.83,17.7,65.83
511,5.99,11.81,0.16,32.73,69.75,1.52,9.90,19.1,74.18
512,14.96,7.07,0.14,10.26,71.40,0.18,11.62,27.2,76.01


# Preprocessing

In [170]:
df['IKP'] = np.log1p(df['IKP'])

In [171]:
y = df['IKP']
x = df.drop(['IKP'], axis=1)

In [172]:
skew_df = x.skew().reset_index(name='skew')
skew_df_filtered = skew_df[skew_df['skew'] >= 0.5]
skew_list= skew_df_filtered['index'].to_list()
skew_df_filtered

,index,skew
0,POV,1.610655
1,FOOD,1.067472
2,ELEC,5.844564
3,WATER,1.128186
5,HEALTH,4.981553


In [173]:
for feature in skew_list:
    optimal_lambda = boxcox_normmax(x[feature] + 1)
    
    x[feature] = boxcox1p(x[feature], optimal_lambda)

In [174]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

def preprocessing(x):
    skew_df = x.skew().reset_index(name='skew')
    skew_df_filtered = skew_df[skew_df['skew'] >= 0.5]
    skew_list = skew_df_filtered['index'].to_list()

    for feature in skew_list:
        optimal_lambda = boxcox_normmax(x[feature] + 1)
        x[feature] = boxcox1p(x[feature], optimal_lambda)
    return x

preprocessing_transformer = FunctionTransformer(preprocessing)

# Create a pipeline with the preprocessing step
pipeline = Pipeline([
    ('preprocessing', preprocessing_transformer)
])

# Save the preprocessing pipeline
with open('preprocessing_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)
print("Preprocessing pipeline saved as 'preprocessing_pipeline.pkl'")

Preprocessing pipeline saved as 'preprocessing_pipeline.pkl'


# Model Development

In [175]:
from sklearn.base import clone
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)

def cross_validation(x, y, model, params=None, cv=cv, feature_importance=None):
    x = x.copy()
    y = y.copy()
    
    list_rmse_val = []
    list_r2_val = []
    list_rmse_train = []
    list_r2_train = []
    best_model = None
    best_rmse_val = float('inf')
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(x, y)):
        x_train, y_train = x.iloc[train_idx], y.iloc[train_idx] 
        x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]
        
        if params is None:
            model_instance = clone(model)
        else:
            model_instance = clone(model(**params))
        
        model_instance.fit(x_train, y_train)
        
        val_pred = model_instance.predict(x_val)
        train_pred = model_instance.predict(x_train)
        
        rmse_val = np.sqrt(mean_squared_error(val_pred, y_val))
        r2_val = r2_score(val_pred, y_val)
        list_rmse_val.append(rmse_val)
        list_r2_val.append(r2_val)
        
        rmse_train = np.sqrt(mean_squared_error(train_pred, y_train))
        r2_train = r2_score(train_pred, y_train)
        list_rmse_train.append(rmse_train)
        list_r2_train.append(r2_train)
        
        print(f'FOLD {fold+1} | RMSE VAL: {rmse_val} | RMSE TRAIN: {rmse_train}')
        print(f'FOLD {fold+1} | R2 VAL: {r2_val} | R2 TRAIN: {r2_train}')
        
        if rmse_val < best_rmse_val:
            best_rmse_val = rmse_val
            best_model = model_instance
    
    print(f'AVG RMSE VAL: {np.mean(list_rmse_val)} | AVG RMSE TRAIN: {np.mean(list_rmse_train)}')
    print(f'AVG R2 VAL: {np.mean(list_r2_val)} | AVG R2 TRAIN: {np.mean(list_r2_train)}')
    
    df_importances = None
    if hasattr(model_instance, 'feature_importances_') or hasattr(model_instance, 'coef_'):
        if hasattr(model_instance, 'feature_importances_'):
            feature_importances = model_instance.feature_importances_
        elif hasattr(model_instance, 'coef_'):
            feature_importances = model_instance.coef_
        
        df_importances = pd.DataFrame({'Feature': x.columns, 'Importance': feature_importances})
        df_importances.sort_values(by='Importance', ascending=False, inplace=True)
    else:
        print("Feature importances not available for this model.")
    
    return best_model, df_importances

# Model Non Linear

In [176]:
xgb_params = {'n_estimators': 5000,
              'random_state': 40,
              'learning_rate': 0.023290207288430613, 
              'gamma': 0.0047549701247300334, 
              'subsample': 0.958891467339551, 
              'colsample_bytree': 0.502879141498819, 
              'max_depth': 15, 
              'min_child_weight': 1, 
              'lambda': 0.20436549730737352, 
              'alpha': 0.0035044512276321328}

best_model, xgb_feature_importances = cross_validation(x, y, XGBRegressor, xgb_params)

FOLD 1 | RMSE VAL: 0.23953961281749325 | RMSE TRAIN: 0.03270553107992549
FOLD 1 | R2 VAL: -1.3300968871076644 | R2 TRAIN: 0.978797836576013
FOLD 2 | RMSE VAL: 0.1835827529234723 | RMSE TRAIN: 0.03251400259754729
FOLD 2 | R2 VAL: -1.847575668962048 | R2 TRAIN: 0.9838624064751164
FOLD 3 | RMSE VAL: 0.1740254815318403 | RMSE TRAIN: 0.03177263838415682
FOLD 3 | R2 VAL: 0.07354388021344871 | R2 TRAIN: 0.9846102599154319
FOLD 4 | RMSE VAL: 0.20649291581826199 | RMSE TRAIN: 0.0319021858086636
FOLD 4 | R2 VAL: -0.20196165157738255 | R2 TRAIN: 0.9811825620518179
FOLD 5 | RMSE VAL: 0.18270137518490182 | RMSE TRAIN: 0.033849712510332614
FOLD 5 | R2 VAL: -1.6932119514463322 | R2 TRAIN: 0.9823068366317849
FOLD 6 | RMSE VAL: 0.21451142406022614 | RMSE TRAIN: 0.03186737006849857
FOLD 6 | R2 VAL: -0.4558928987808515 | R2 TRAIN: 0.9842974262316232
FOLD 7 | RMSE VAL: 0.16567253523440537 | RMSE TRAIN: 0.03303275209299713
FOLD 7 | R2 VAL: -0.21655940895566017 | R2 TRAIN: 0.982520515789407
FOLD 8 | RMSE VA

# Hyperparamater

In [177]:
tune = False

xgb

In [178]:

def objective(trial):
    param = {
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'gamma' : trial.suggest_float('gamma', 1e-5, 0.5, log=True),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True), 
    }
    scores_rmse_val = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(x, y)):
        X_train = x.iloc[train_idx]
        y_train = y.iloc[train_idx]
        
        X_val = x.iloc[val_idx]
        y_val = y.iloc[val_idx]
        
        model_xgb = XGBRegressor(**param)
        model_xgb.fit(X_train, y_train)
        preds = model_xgb.predict(X_val)
        
        score = np.sqrt(mean_squared_error(preds,y_val))
        scores_rmse_val.append(score)
    
    return np.mean(scores_rmse_val)
if tune:
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=200)


    best_params = study.best_params

## Model with Hyperparamater

xgb

In [179]:
# Model with tuned hyperparameters
xgb_params_tune = {'learning_rate': 0.03243310688668244,
 'gamma': 0.00013359103368449678,
 'subsample': 0.9537324184642,
 'colsample_bytree': 0.49142254237112015,
 'max_depth': 3,
 'min_child_weight': 9,
 'lambda': 0.1282706050863809,
 'alpha': 0.024842783553593806}

best_model, xgb_feature_importances = cross_validation(x, y, XGBRegressor, xgb_params_tune)

FOLD 1 | RMSE VAL: 0.22383305609594184 | RMSE TRAIN: 0.14275936377350273
FOLD 1 | R2 VAL: -0.3137209066956279 | R2 TRAIN: 0.24069332285158174
FOLD 2 | RMSE VAL: 0.17372195905193452 | RMSE TRAIN: 0.14409008556630762
FOLD 2 | R2 VAL: -1.4335103916139587 | R2 TRAIN: 0.4667701640371973
FOLD 3 | RMSE VAL: 0.16224444326017004 | RMSE TRAIN: 0.14392362061207709
FOLD 3 | R2 VAL: -0.029479569490794777 | R2 TRAIN: 0.4496720286356368
FOLD 4 | RMSE VAL: 0.2065172175012842 | RMSE TRAIN: 0.14340011645490144
FOLD 4 | R2 VAL: -0.02620019437712906 | R2 TRAIN: 0.3010269781474546
FOLD 5 | RMSE VAL: 0.172677281676404 | RMSE TRAIN: 0.14566626426658727
FOLD 5 | R2 VAL: -2.5930288572675835 | R2 TRAIN: 0.4505005505200419
FOLD 6 | RMSE VAL: 0.1982357936535213 | RMSE TRAIN: 0.13998704500163991
FOLD 6 | R2 VAL: -0.6784250201847248 | R2 TRAIN: 0.5150629738718218
FOLD 7 | RMSE VAL: 0.16324225261322972 | RMSE TRAIN: 0.14659265130086016
FOLD 7 | R2 VAL: -0.009625281054391532 | R2 TRAIN: 0.39796063489484934
FOLD 8 | R

# Feature Importances

In [180]:
xgb_feature_importances

,Feature,Importance
0,POV,0.353952
5,HEALTH,0.135265
2,ELEC,0.118553
3,WATER,0.117744
1,FOOD,0.102559
6,SCHOOL,0.089588
4,LIFE,0.044721
7,STUNTING,0.037616


In [181]:
import pickle

# Save the model as a .pkl file
with open('xgb_tuned_model.pkl', 'wb') as file:
    pickle.dump(best_model, file)
print("Tuned model saved as 'xgb_tuned_model.pkl'")

Tuned model saved as 'xgb_tuned_model.pkl'
